# <u> LGA Mapping </u>
## Last updated: 2021-03-08 | Created by: Rebecca Jan

### Please note you may need to create a separate Python environment to run this script, I had to create geo_env instance for this to work

Set up notebook

In [5]:
import geopandas as gpd
from geopandas.tools import sjoin
import numpy as np
import pandas as pd
import shapely
from shapely.geometry import Point,  Polygon

Load .csv and .shp file to be mapped <br></br>
LGA lookup  sourced from - <br></br>
https://data.gov.au/dataset/ds-dga-bdf92691-c6fe-42b9-a0e2-a4cd716fa811/details

In [10]:
map_url = "https://github.com/rjanhealth/lga_mapping/blob/c89b4674baf7ac9b2abb11d10e162a64dbb821f5/data/vic_lga_polygon_shp/VIC_LGA_POLYGON_SHP.shp"
vic_lga = gpd.read_file(map_url)
header_list = ["testdate", "testtype", "lga_old", "metrorural_old", "addressline1",
                     "addressline2", "fullcleanaddress", "streetname", "suburb", "state", "postcode", 
                     "rawpostcode", "longitude", "latitude"]
sitrep = pd.read_csv("sitrep.csv", header = None, names = header_list, sep = ",")
lkp_url = "https://github.com/rjanhealth/lga_mapping/blob/c89b4674baf7ac9b2abb11d10e162a64dbb821f5/data/lga_lookup.csv"
lga_lookup = pd.read_csv(lkup_url, sep = ",")
sitrep["TestDate"]= pd.to_datetime(sitrep["TestDate"])

HTTPError: HTTP Error 404: Not Found

Assign points to .csv data

In [ ]:
geometry = [Point(xy) for xy in zip(sitrep["longitude"], sitrep["'atitude"])]
geo_df = gpd.GeoDataFrame(sitrep, crs="EPSG:4326", geometry=geometry)

Run inner join to pull in .shp data based on point

In [ ]:
vic_lga = vic_lga.to_crs(crs="EPSG:4326")
pointInPolys = sjoin(geo_df, vic_lga, how="left", op="within")

Pivot data so interstate is separated out of state

In [ ]:
pointInPolys["Interstate"] = np.where(pointInPolys['LGA']=="Interstate", "Y", "N")

Fill in empty column values for LGA_NAME & standardise to upper

In [ ]:
pointInPolys.loc[pointInPolys["LGA_NAME"].isnull(),"LGA_NAME"] = pointInPolys["LGA"]
pointInPolys['LGA_NAME'] = pointInPolys['LGA_NAME'].map(lga_lookup.set_index('LOOKUP COL')['RETURN THIS'])
pointInPolys.columns= pointInPolys.columns.str.upper()

Check data  length to ensure all rows are retained

In [ ]:
#print(len(geo_df)-len(pointInPolys))
#pointInPolys.head(50)
pointInPolys.info()
#geo_df[~geo_df['RecordID'].isin(pointInPolys ['RecordID'])]
#pointInPolys.LGA_NAME.unique()

Change data types

In [ ]:
pointInPolys['RAWPOSTCODE'] = pointInPolys['RAWPOSTCODE'].astype('Int64')
pointInPolys['POSTCODE'] = pointInPolys['POSTCODE'].astype('Int64')

Drop columns as needed

In [ ]:
pointInPolys.drop(pointInPolys.columns[[15, 16, 17, 18, 22]], axis = 1, inplace = True)

Export and write to csv

In [ ]:
#pointInPolys.to_csv("lga_mapped.csv", index=False)